### Imports

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd

### Data ingest

In [2]:
df = pd.read_csv('./data_in/data_kershaw_regularseason_pitches_2008to2021.csv')

print(df.shape)
print(df.columns)
df.head(2)

(37071, 92)
Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
  

C:\Users\m85302\Anaconda3b\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (43,87,88) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,FF,2008-05-25,93.1,0.23,5.86,"Kershaw, Clayton",435401,477132,grounded_into_double_play,hit_into_play,...,1,1,1,1,1,NaN,NaN,NaN,0.083,-0.604
1,NaN,2008-05-25,NaN,NaN,NaN,"Kershaw, Clayton",435401,477132,NaN,foul,...,0,0,0,0,0,NaN,NaN,NaN,0.000,0.000


### Data clean1

In [3]:
df['game_date'] = pd.to_datetime(df['game_date'])
df = df.sort_values(by='game_date')
df = df.reset_index(drop=True)
display(df.head(3))
display(df.tail(3))

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,FF,2008-05-25,93.1,0.23,5.86,"Kershaw, Clayton",435401,477132,grounded_into_double_play,hit_into_play,...,1,1,1,1,1,NaN,NaN,NaN,0.083,-0.604
1,CU,2008-05-25,76.5,-0.20,5.95,"Kershaw, Clayton",136267,477132,NaN,blocked_ball,...,2,2,2,2,2,NaN,NaN,NaN,0.000,0.066
2,CU,2008-05-25,73.5,-0.12,6.31,"Kershaw, Clayton",407886,477132,NaN,called_strike,...,1,1,1,1,1,NaN,NaN,NaN,0.000,-0.023


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
37068,SL,2021-10-01,86.9,1.37,6.18,"Kershaw, Clayton",592885,477132,NaN,ball,...,0,0,0,0,0,Strategic,Standard,200.0,0.00,0.011
37069,FF,2021-10-01,90.3,1.30,6.17,"Kershaw, Clayton",541645,477132,field_out,hit_into_play,...,0,0,0,0,0,Infield shift,Standard,175.0,0.03,-0.273
37070,CU,2021-10-01,75.1,0.98,6.28,"Kershaw, Clayton",649966,477132,NaN,called_strike,...,1,0,1,0,1,Infield shift,Standard,336.0,0.00,-0.088


### Prepare clean Xy data for future

In [4]:
def select_pitch_type_target(df_in, target_pitch_in):
    """
        Tags the pitch type given in 'target_pitch_in' as '1', 
        and all other pitch types as '0', in preparation
        for binary classification step
    """
    ### turn pitch_type into binary (to start with). Just: is FF or is not:
    df_out = df_in.copy()
    df_out['pitch_bin'] = 0
    # df_out.loc[df_out.pitch_type == 'FF', 'FF_bin'] = 1
    df_out.loc[df_out.pitch_type == target_pitch_in, 'pitch_bin'] = 1
    return df_out

In [5]:
print('NaNs in pitch_type target variable: ', df['pitch_type'].isnull().sum())
print(df['pitch_type'].isnull().sum() / df.shape[0])

NaNs in pitch_type target variable:  568
0.015321949772059022


In [6]:
# Feature selection1:

print(df.shape)

#df = df[['pitch_type', 'balls', 'strikes', 'game_year', 'batter', 'pitch_number',
#       'at_bat_number', 'home_score', 'away_score', 'if_fielding_alignment',
#       'of_fielding_alignment', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up',
#       'pitch_bin']]

df = df[['pitch_type', 'balls', 'strikes', 'game_year', 'batter', 'pitch_number',
       'at_bat_number', 'home_score', 'away_score', 'if_fielding_alignment',
       'of_fielding_alignment', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up']]

print(df.shape)

(37071, 92)
(37071, 15)


In [7]:
def remove_nans(df_in, col_list):
    """
        Deletes rows in df_in that contain NaNs in the columns in col_list
    """
    df_out = df_in.dropna(subset=[col_list])
    return df_out

In [8]:
remove_nans_from_cols = 'pitch_type'
target_pitch_class = 'FF' #'CH' #'CU' #'SL' #'FF' #'SL' #'CU' #'FF' #'CU' #'FF'

df = remove_nans(df, remove_nans_from_cols)
df = select_pitch_type_target(df, target_pitch_class)
print(df.shape)

(36503, 16)


#### Prepare Xy for ML

In [9]:
# Data prep for ML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# ML algos
from sklearn.decomposition import PCA

In [10]:
def calc_class_imbal(y_in):
    """
    Insert Function Description
    """
    df_ytrain = pd.DataFrame(y_in)
    df_ytrain.columns = ['y']
    df_y_0 = df_ytrain[df_ytrain['y'] == 0]
    df_y_1 = df_ytrain[df_ytrain['y'] == 1]
    num_train0_out = df_y_0.shape[0]
    num_train1_out = df_y_1.shape[0]
    frac_0 = num_train0_out / num_train1_out
    print(" ")
    print('num_train0_out: ', num_train0_out)
    print('num_train1_out: ', num_train1_out)
    print('frac_0: (num_train0_out / num_train1_out): ', frac_0)
    
    return frac_0

def standardize_data(X_train_in, X_test_in):
    """
    Insert Function Description
    """
    sc = StandardScaler()
    sc.fit(X_train_in)
    X_train_std_out = sc.transform(X_train_in)
    X_test_std_out = sc.transform(X_test_in)
    
    return X_train_std_out, X_test_std_out

def pca_dim_red(X_train_in, X_test_in):
    
    pca = PCA(n_components=2)
    X_train_pca_out = pca.fit_transform(X_train_in)
    pca.fit_transform(X_train_in)
    X_test_pca_out = pca.transform(X_test_in)
    
    return X_train_pca_out, X_test_pca_out

def create_X_y_dfs(df_in): 
    """
        Hard coded for this specific dataset (generalize later)
    """
    df_y_out = pd.DataFrame(df_in['pitch_bin'])
    df_X_out = df_in.drop(columns=['pitch_type', 'pitch_bin'], axis=1)
    return df_X_out, df_y_out

def create_numeric_and_onehot_features(df_in):
    """
    Insert Function Description
    """    
    df_out = pd.get_dummies(df_in)
    return df_out

def handle_nans(df_in, nan_strategy_in):
    """
    Insert Function Description
    """   
    df_out = df_in
    if (nan_strategy_in == 'fill_zero'):
        df_out = df_in.fillna(0)
    return df_out

def create_ml_data_splits(df_X_in, df_y_in, test_size_in):
    """
    Insert Function Description
    """
    X = df_X_in.values
    y = df_y_in.values
    X_train_out, X_test_out, y_train_out, y_test_out = train_test_split(X, y, test_size=test_size, 
                                                                        stratify=y) # random_state=1
    return X_train_out, X_test_out, y_train_out, y_test_out

In [11]:
test_size = 0.3
nan_strategy = 'fill_zero'

In [12]:
df_X, df_y = create_X_y_dfs(df)
df_X = create_numeric_and_onehot_features(df_X)
df_X = handle_nans(df_X, nan_strategy)

X_train, X_test, y_train, y_test = create_ml_data_splits(df_X, df_y, test_size) 

#frac_class0 = calc_class_imbal(y_train)
#X_train_std, X_test_std = standardize_data(X_train, X_test)
#X_train_pca, X_test_pca = pca_dim_red(X_train_std, X_test_std)

In [13]:
print(df_X.shape, df_y.shape)

(36503, 18) (36503, 1)


In [14]:
display(df_X.head(3))
display(df_y.head(3))

,balls,strikes,game_year,batter,pitch_number,at_bat_number,home_score,away_score,on_3b,on_2b,on_1b,outs_when_up,if_fielding_alignment_Infield shift,if_fielding_alignment_Standard,if_fielding_alignment_Strategic,of_fielding_alignment_Extreme outfield shift,of_fielding_alignment_Standard,of_fielding_alignment_Strategic
0,2,1,2008,435401,4,17,1,1,0.0,0.0,453895.0,0,0,0,0,0,0,0
1,1,2,2008,136267,5,41,2,2,0.0,405395.0,407886.0,1,0,0,0,0,0,0
2,0,0,2008,407886,1,23,1,1,0.0,0.0,0.0,1,0,0,0,0,0,0


,pitch_bin
0,1
1,0
2,0


In [16]:
print(df.shape, df_X.shape, df_y.shape)

(36503, 16) (36503, 18) (36503, 1)


In [18]:
#df_X.to_csv('data_X_cleaned_kersh2008to2021.csv', index=False)
#df_y.to_csv('data_y_cleaned_FF1_kersh2008to2021.csv', index=False)

In [19]:
df_y.head(2)

,pitch_bin
0,1
1,0


In [20]:
df_Xy = df_X.copy()
df_Xy['pitch_bin'] = df_y.values

In [21]:
df_Xy.head(3)

,balls,strikes,game_year,batter,pitch_number,at_bat_number,home_score,away_score,on_3b,on_2b,on_1b,outs_when_up,if_fielding_alignment_Infield shift,if_fielding_alignment_Standard,if_fielding_alignment_Strategic,of_fielding_alignment_Extreme outfield shift,of_fielding_alignment_Standard,of_fielding_alignment_Strategic,pitch_bin
0,2,1,2008,435401,4,17,1,1,0.0,0.0,453895.0,0,0,0,0,0,0,0,1
1,1,2,2008,136267,5,41,2,2,0.0,405395.0,407886.0,1,0,0,0,0,0,0,0
2,0,0,2008,407886,1,23,1,1,0.0,0.0,0.0,1,0,0,0,0,0,0,0


In [22]:
df_Xy.to_csv('data_Xy_cleaned_FF1_kersh2008to2021.csv', index=False)